# 4.x 高维因果推断模板：Double ML + 因果森林（可运行示例）

本 Notebook 提供**最小可运行模板**：
- **Double Machine Learning (DML)**：适合高维数据（L1/L2 正则 + 交叉拟合）
- **Causal Forest**：捕捉异质性治疗效应（CATE/ITE）

> 默认用**可运行的高维模拟数据**演示完整流程；把数据替换为你的医学数据即可（如 EHR/组学）。

## 0. 环境准备（如已安装可跳过）

```bash
pip install -U numpy pandas scikit-learn matplotlib econml
# （可选）若你想用 causalml 的因果森林：
pip install -U causalml
```
> 说明：本模板使用 **econml** 的 `LinearDML` 与 `CausalForestDML`；如需 `causalml` 版本，请参考文末的可选代码块。

## 1. 生成/载入数据（高维场景）
我们构造一个**高维有混杂且存在异质性**的二元处理示例：
$T = \mathbb{1}(g(X) + \epsilon > 0)$；$Y = f(X) + \tau(X)\,T + \eta$，其中 $\tau(X)$ 随部分特征而变。

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(2025)

def make_highdim_data(n=2000, p=100, rho=0.3):
    # 相关协方差矩阵（AR(1)）
    Sigma = rho ** np.abs(np.subtract.outer(np.arange(p), np.arange(p)))
    X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=n)
    # 真实影响治疗/结局的少数变量索引
    idx_t = np.array([0, 1, 2, 5, 9])
    idx_y = np.array([0, 3, 4, 7, 11])
    idx_tau = np.array([2, 6, 8])  # 异质性来源

    # 治疗赋值（含混杂）
    g = 0.6*X[:, idx_t].sum(axis=1)
    ps = 1/(1+np.exp(-g))
    T = (np.random.rand(n) < ps).astype(int)

    # 结局基线项 + 异质性治疗效应 tau(X)
    f = 0.5*X[:, idx_y].sum(axis=1)
    tau = 1.0 + 0.8*np.tanh(X[:, idx_tau].sum(axis=1))  # 在 [-? , ?] 范围平滑变化
    Y = f + tau*T + np.random.normal(0, 1.0, size=n)

    cols = [f"x{i}" for i in range(p)]
    df = pd.DataFrame(X, columns=cols)
    df['T'] = T
    df['Y'] = Y
    return df

df = make_highdim_data()
df.head()

## 2. LASSO（L1）变量选择 + 倾向评分（PS）建模
在高维下，先用 **L1** 做稀疏选择，帮助构建稳健的 PS 模型。

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

X = df.drop(columns=['T','Y'])
T = df['T'].values
Y = df['Y'].values

X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.3, random_state=42)

# L1-Logit 进行变量选择（saga 支持 L1）
lasso_logit = LogisticRegressionCV(Cs=10, penalty='l1', solver='saga', cv=5, max_iter=5000, n_jobs=-1)
lasso_logit.fit(X_train, T_train)

coef = lasso_logit.coef_.ravel()
selected = X.columns[coef != 0]
print('Selected confounders (non-zero in L1-Logit):', list(selected)[:20], '...')

# 用全部特征重拟合 PS，或仅用 selected 再拟合一次（两种策略都常见）
ps_model = LogisticRegressionCV(Cs=10, penalty='l2', solver='lbfgs', cv=5, max_iter=5000, n_jobs=-1)
ps_model.fit(X, T)
ps = ps_model.predict_proba(X)[:,1]

print('PS AUC (in-sample):', round(roc_auc_score(T, ps), 3))

# PS 分布图（治疗/对照各一张，便于报告复用）
plt.figure()
plt.hist(ps[T==1], bins=40, alpha=0.7, density=True, label='T=1')
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('PS Distribution (Treated)')
plt.legend()
plt.show()

plt.figure()
plt.hist(ps[T==0], bins=40, alpha=0.7, density=True, label='T=0')
plt.xlabel('Propensity Score')
plt.ylabel('Density')
plt.title('PS Distribution (Control)')
plt.legend()
plt.show()

## 3. Double Machine Learning（DML）估计 ATE/CATE
使用 **econml** 的 `LinearDML`（离散处理）。

In [ ]:
from econml.dml import LinearDML
from sklearn.linear_model import LassoCV

dml = LinearDML(
    model_y=LassoCV(cv=5, n_jobs=-1),
    model_t=LogisticRegressionCV(Cs=10, cv=5, max_iter=5000, n_jobs=-1),
    discrete_treatment=True,
    cv=3,
    random_state=42
)
dml.fit(Y, T, X=X)
cate_dml = dml.effect(X)
ate_dml = float(np.mean(cate_dml))
print('DML ATE (mean CATE):', round(ate_dml, 4))

# CATE 分布（单图）
import matplotlib.pyplot as plt
plt.figure()
plt.hist(cate_dml, bins=40)
plt.xlabel('Estimated CATE (DML)')
plt.ylabel('Count')
plt.title('Distribution of CATE (DML)')
plt.show()

## 4. 因果森林（CausalForestDML）估计 CATE
使用 **econml** 的 `CausalForestDML`（适合非线性/异质性）。

In [ ]:
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor

cf = CausalForestDML(
    model_t=LogisticRegressionCV(Cs=5, cv=3, max_iter=3000, n_jobs=-1),
    model_y=RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    discrete_treatment=True,
    n_estimators=500,
    random_state=42
)
cf.fit(Y, T, X=X)
cate_cf = cf.effect(X)
ate_cf = float(np.mean(cate_cf))
print('CausalForest ATE (mean CATE):', round(ate_cf, 4))

plt.figure()
plt.hist(cate_cf, bins=40)
plt.xlabel('Estimated CATE (CausalForest)')
plt.ylabel('Count')
plt.title('Distribution of CATE (CausalForest)')
plt.show()

## 5. 异质性切片：高/低 CATE 亚组对比
将个体按 CATE 分位数分组，输出分层平均治疗效应，辅助精准医疗解读。

In [ ]:
def summarize_by_quantiles(cate, q=(0.1, 0.5, 0.9)):
    qs = np.quantile(cate, q)
    return {f'Q{int(p*100)}': float(v) for p, v in zip(q, qs)}

print('DML CATE quantiles:', summarize_by_quantiles(cate_dml))
print('CF  CATE quantiles:', summarize_by_quantiles(cate_cf))

# 取上下 20% 分组，估计分层均值（演示用途；实际应进一步做统计检验/误差估计）
q_low, q_high = np.quantile(cate_cf, [0.2, 0.8])
low_idx  = cate_cf <= q_low
high_idx = cate_cf >= q_high
print('Mean CATE (CF) - bottom 20%:', round(float(cate_cf[low_idx].mean()), 4))
print('Mean CATE (CF) - top    20%:', round(float(cate_cf[high_idx].mean()), 4))

## 6. （可选）IPTW 作为基线对照
用前文 PS 构造**稳定化权重**，估计加权风险差/均值差，作为 DML/CF 的 sanity check。

In [ ]:
p_treat = T.mean()
sw = np.where(T==1, p_treat/ps, (1-p_treat)/(1-ps))
# 连续结局：加权均值差
y_t = np.average(Y[T==1], weights=sw[T==1])
y_c = np.average(Y[T==0], weights=sw[T==0])
print('IPTW weighted difference (T-C):', round(float(y_t - y_c), 4))

## 7. （可选）使用 `causalml` 的因果森林
如果你更熟悉 `causalml`，可以使用其 `CausalForest`：

```python
from causalml.inference.tree import CausalForest
cf = CausalForest()
cf.fit(X=X.values, treatment=T, y=Y)
ite = cf.predict(X.values)
```
> 注意：API 版本可能有细微区别，请以本地 `causalml` 文档为准。